In [17]:
pip install -U vllm --pre --extra-index-url https://wheels.vllm.ai/nightly


Looking in indexes: https://pypi.org/simple, https://wheels.vllm.ai/nightly
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.6/394.6 MB 19.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: vllm
    Found existing installation: vllm 0.9.2.dev5+g77f0d465d
    Uninstalling vllm-0.9.2.dev5+g77f0d465d:
      Successfully uninstalled vllm-0.9.2.dev5+g77f0d465d

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install gradio


Found existing installation: gradio 5.33.1
Uninstalling gradio-5.33.1:
  Successfully uninstalled gradio-5.33.1
  Using cached gradio-5.33.1-py3-none-any.whl.metadata (16 kB)
Using cached gradio-5.33.1-py3-none-any.whl (54.2 MB)

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Streaming response from the model

In [15]:
import gradio as gr
from openai import OpenAI
import re
import time

client = OpenAI(api_key="EMPTY", base_url="http://localhost:8000/v1")

SYSTEM_PROMPT = """<s>[SYSTEM_PROMPT]system_prompt

A user will ask you to solve a task. You should first draft your thinking process (inner monologue) until you have derived the final answer. Afterwards, write a self-contained summary of your thoughts.

<think>
Your thoughts or draft, like working through an exercise on scratch paper.
</think>

Here, provide a concise summary that reflects your reasoning and presents a clear final answer to the user.

Problem:
[/SYSTEM_PROMPT]"""

# Streaming logic with stop control
def debug_faulty_logic_stream(faulty_proof, stop_signal):
    stop_signal["stop"] = False  # Reset stop flag

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"Here is a flawed logic or math proof. Can you debug it step-by-step?\n\n{faulty_proof}"}
    ]

    try:
        response = client.chat.completions.create(
            model="mistralai/Magistral-Small-2506",
            messages=messages,
            stream=True,
            temperature=0.7,
            top_p=0.95,
            max_tokens=2048
        )

        buffer = ""
        for chunk in response:
            if stop_signal.get("stop"):
                break
            delta = chunk.choices[0].delta
            if hasattr(delta, "content") and delta.content:
                buffer += delta.content
                filtered = re.sub(r"<think>.*?</think>", "", buffer, flags=re.DOTALL).strip()
                yield filtered
            time.sleep(0.02)
    except Exception as e:
        yield f"Error: {str(e)}"

# Set stop flag when stop button is clicked
def stop_streaming(stop_signal):
    stop_signal["stop"] = True
    return gr.Textbox.update(value="Stopped.")


with gr.Blocks() as demo:
    gr.Markdown("## Chain-of-Logic Debugger (Streaming via Magistral + vLLM)")

    input_box = gr.Textbox(
        label="Paste Your Faulty Logic or Proof",
        lines=8,
        placeholder="e.g., Assume x = y, then x² = xy..."
    )
    output_box = gr.Textbox(label="Corrected Reasoning (Streaming Output)")

    submit_btn = gr.Button("Submit")
    stop_btn = gr.Button("Stop")

    stop_flag = gr.State({"stop": False})

    submit_btn.click(
        fn=debug_faulty_logic_stream,
        inputs=[input_box, stop_flag], 
        outputs=output_box
    )

    stop_btn.click(
        fn=stop_streaming,
        inputs=stop_flag,
        outputs=output_box
    )

if __name__ == "__main__":
    demo.launch(share=True, inbrowser=True, debug=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://859ccc19cc37ca4b9c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://859ccc19cc37ca4b9c.gradio.live


# Streaming only final summary

In [13]:
import gradio as gr
from openai import OpenAI
import re

client = OpenAI(api_key="EMPTY", base_url="http://localhost:8000/v1")

SYSTEM_PROMPT = """<s>[SYSTEM_PROMPT]system_prompt

A user will ask you to solve a task. You should first draft your thinking process (inner monologue) until you have derived the final answer. Afterwards, write a self-contained summary of your thoughts.

<think>
Your thoughts or draft, like working through an exercise on scratch paper.
</think>

Here, provide a concise summary that reflects your reasoning and presents a clear final answer to the user.

Problem:
[/SYSTEM_PROMPT]"""

def extract_summary_after_think(text):
    match = re.search(r"</think\s*>(.*)", text, flags=re.IGNORECASE | re.DOTALL)
    return match.group(1).strip() if match else "No summary found after </think>"

def debug_faulty_logic_stream(faulty_proof):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"Here is a flawed logic or math proof. Can you debug it step-by-step?\n\n{faulty_proof}"}
    ]

    try:
        response = client.chat.completions.create(
            model="mistralai/Magistral-Small-2506",
            messages=messages,
            stream=True,
            temperature=0.7,
            top_p=0.95,
            max_tokens=2048
        )

        buffer = ""
        for chunk in response:
            delta = chunk.choices[0].delta
            if hasattr(delta, "content") and delta.content:
                buffer += delta.content

        return extract_summary_after_think(buffer)

    except Exception as e:
        return f"Error: {str(e)}"

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("## Chain-of-Logic Debugger (Magistral + vLLM)")

    input_box = gr.Textbox(
        label="Paste Your Faulty Logic or Proof",
        lines=8,
        placeholder="e.g., Assume x = y, then x² = xy..."
    )
    output_box = gr.Textbox(label="Final Summary")

    submit_btn = gr.Button("Submit")

    submit_btn.click(
        fn=debug_faulty_logic_stream,
        inputs=input_box,
        outputs=output_box
    )

if __name__ == "__main__":
    demo.launch(share=True, inbrowser=True, debug=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://f89405655b0f4175ff.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://f89405655b0f4175ff.gradio.live
